<a href="https://colab.research.google.com/github/ak9250/impersonator/blob/master/impersonator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/svip-lab/impersonator.git

In [0]:
cd impersonator/

In [0]:
!pip install -r requirements.txt

In [0]:
cd thirdparty/neural_renderer

In [0]:
!python setup.py install

In [0]:
cd ../..

In [0]:
!wget -O assets/pretrains.zip https://1drv.ws/u/s!AjjUqiJZsj8whLNw4QyntCMsDKQjSg?e=L77Elv

In [0]:
cd assets

In [0]:
!unzip pretrains.zip

In [0]:
cd ..

In [0]:
!wget -O outputs/checkpoints.zip https://1drv.ws/u/s!AjjUqiJZsj8whLNyoEh67Uu0LlxquA?e=dkOnhQ

In [0]:
cd outputs

In [0]:
!unzip checkpoints.zip

In [0]:
cd ..

In [0]:
!wget -O assets/samples.zip https://1drv.ws/u/s!AjjUqiJZsj8whLNz4BqnSgqrVwAXoQ?e=bC86db

In [0]:
cd assets

In [0]:
!unzip samples.zip

In [0]:
cd ..

In [0]:
%%bash
python run_imitator.py --gpu_ids 0 --model imitator --output_dir ./outputs/results/  \
    --src_path      ./assets/src_imgs/internet/men1_256.jpg    \
    --tgt_path      ./assets/samples/refs/iPER/024_8_2    \
    --bg_ks 7   --ft_ks 3 \
    --has_detector  --post_tune --front_warp \
    --save_res

In [0]:
%%bash
python run_swap.py --gpu_ids 0 --model imitator --output_dir ./outputs/results/  \
    --src_path      ./assets/src_imgs/imper_A_Pose/024_8_2_0000.jpg    \
    --tgt_path      ./assets/src_imgs/fashion_man/Sweatshirts_Hoodies-id_0000680701_4_full.jpg    \
    --bg_ks 13  --ft_ks 3 \
    --has_detector  --post_tune  --front_warp --swap_part body  \
    --save_res

In [0]:
%%bash
python run_view.py --gpu_ids 0 --model viewer --output_dir ./outputs/results/  \
--src_path      ./assets/src_imgs/internet/women1_256.jpg    \
--bg_ks 13  --ft_ks 3 \
--has_detector  --post_tune --front_warp --bg_replace \
--save_res

In [0]:
!ffmpeg -f image2 -i /content/impersonator/outputs/results/imgs/%*.jpg -c:v libx264 -pix_fmt yuv420p out.mp4 -y

In [0]:
!ffmpeg -f image2 -i /content/impersonator/outputs/results/imitators/gt_%04d.jpg -c:v libx264 -pix_fmt yuv420p out2.mp4 -y

In [0]:
!ffmpeg -f image2 -i /content/impersonator/outputs/results/imitators/pred_%04d.jpg -c:v libx264 -pix_fmt yuv420p out3.mp4 -y

In [0]:
!ffmpeg -i out2.mp4 -filter_complex "[0:v] palettegen" palette.png -y

In [0]:
!ffmpeg -i out2.mp4 -i palette.png -filter_complex "[0:v][1:v] paletteuse" endresult.gif -y

In [0]:
!ffmpeg -i out3.mp4 -filter_complex "[0:v] palettegen" palette2.png -y

In [0]:
!ffmpeg -i out3.mp4 -i palette2.png -filter_complex "[0:v][1:v] paletteuse" endresult2.gif -y

In [0]:
!ffmpeg -i endresult.gif -i endresult2.gif -filter_complex hstack outputstack2.gif -y

In [0]:
!ffmpeg -i outputstack2.gif -f mp4 -c:v libx264 -pix_fmt yuv420p combined.mp4